<a href="https://colab.research.google.com/github/4rldur0/whyfi/blob/xeoyeon/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab에서 돌려보기

## 1. google drive 연동
데이터셋 가져오기

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/RAG

In [ ]:
!pip install transformers chromadb langchain langchain_community langchain-chroma langchain-huggingface

## 2. 사용할 dataset embedding, vector db 설정
- dataset :
    - "한국은행 경제금융용어 700선.pdf"=> 데이터 파싱하여 csv 파일 사용(cleaned_word_dict.csv)
    - "2024 한권으로 OK 주식과 세금.pdf"
    - "알고 싶어요? 주식시장.pdf"
- embedding : "dragonkue/BGE-m3-ko"
- vector DB : chromaDB

### pdf -> chunk


In [ ]:
!pip install pymupdf

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# 1. 여러 개의 PDF 파일 로드
pdf_paths = ["/content/drive/MyDrive/Colab Notebooks/RAG/dataset/[2024 한권으로 OK 주식과 세금].pdf", "/content/drive/MyDrive/Colab Notebooks/RAG/dataset/[알고 싶어요 주식시장].pdf"]  # PDF 파일 리스트
documents = []
for pdf in pdf_paths:
    loader = PyMuPDFLoader(pdf)
    documents.extend(loader.load())

# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# 3. 임베딩 모델 설정 및 저장
embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" # 데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
)

vectorstore.add_documents(split_docs) # vectorstore에 저장

### csv file

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/RAG/dataset/cleaned_word_dict.csv'
data = pd.read_csv(file_path)

In [ ]:
from langchain.docstore.document import Document # 데이터의 각 row를 document 객체로 변환하여 저장하기 위함

# vectorDB에 data 추가하는 함수
def add_data_to_vectorstore(data, vectorstore):
  for index, row in data.iterrows(): # row라는 변수에 각 행을 반복적으로 가져옴.
    text = row.get("Content","")
    metadata = row.to_dict() #행 전체를 딕셔너리 형태로 변환
    metadata["source"] = metadata.get("source", f"row_{index}")   # source 필드 추가 (기본값으로 행 번호를 사용하거나 특정 열에서 가져오기)
    document = Document(page_content=text, metadata=metadata) #cocument 객체를 생성
    vectorstore.add_texts([document.page_content],[document.metadata]) # db에 데이터 추가

add_data_to_vectorstore(data, vectorstore)

### 전에 임베딩 해둔 경우

In [ ]:
# 기존 임베딩 결과만 가져오기
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" # 데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
)

## 3. Retriever 및 프롬프트 설정

In [ ]:
# Define a retriever to search in the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 검색 시 가장 관련성 높은 3개의 문서를 반환하라는 뜻

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어나 개념을 최대 3개까지 추천하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

## 4. gemini api 설정 및 chain 구현하기

In [ ]:
!pip install langchain-google-vertexai

In [ ]:
PROJECT_ID = "your_project_id"
REGION = "your_region"

from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# Chain 사용 예시
question = "가동률이란?"
result = chain({"question": question})

print("Answer:", result["answer"])
print("Sources:", result["source_documents"])

# streamlit
1. [ngrok](https://dashboard.ngrok.com/) 에 접속 후 회원가입
2. 로그인 후 뜨는 authtoken 번호를 아래의 코드에 붙여넣기
3. app.py 파일을 생성하여 돌리기


In [ ]:
!pip install langchain-google-vertexai

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
PROJECT_ID = "your_project_id"
REGION = "your_region"

from google.colab import auth
auth.authenticate_user()

In [ ]:
%%writefile app.py

import streamlit as st
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import re
import requests

# 1. embedding model 설정
embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# 2. vector db 설정 (chromaDB 사용)
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정
)

# 3. retriever 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. prompt 설정
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어를 추천하세요.

단, 반드시 한국어로 답변해야 하며, 전체 Answer의 길이가 8줄을 넘어가지 않도록 하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

# 5. LLM 설정 (gemini 사용)
PROJECT_ID = "your_project_id"
REGION = "your_region"
vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

# 6. 최종 chain 설계
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

# 7. 네이버 뉴스 API
CLIENT_ID="satq5UZoVM2fiHsCHrvy"
CLIENT_SECRET="Gj_rU32L3v"

def get_naver_news(query, display=5):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
    }
    params = {
        "query": query,
        "display": display,
        "sort": "sim"  # 유사도 기준 정렬
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        news_items = response.json().get('items', [])
        return news_items[:3]  # 유사도가 높은 3개 뉴스만 반환
    else:
        return []

###### streamlit 설계 #######
st.set_page_config(page_title="금융용어알리미 : WhyFi")

st.markdown(
    """
    <h1 style='color: white; text-align: center;'>금융용어알리미 : WhyFi</h1>
    <h2 style='color: gray; text-align: center; font-size: 20px;'>모르는 금융 용어? 이제 쉽게 찾아보세요!</h2>
    <br>
    """,
    unsafe_allow_html=True,
)


user_question = st.text_input("금융 용어를 검색해보세요.", value="", placeholder="예: 가동률이란?")
if user_question:
    with st.spinner("답변을 생성 중입니다..."):
        result = chain({"question": user_question})

    # 결과 출력
    st.markdown("#### 📖 답변 📖")
    answer_text = result["answer"]
    st.write(answer_text)


    st.markdown("### 📚 참고 출처 📚")
    for doc in result["source_documents"]:
        st.markdown(
            f"<div style='font-size:14px; line-height:1.4;'>- {doc.metadata['source']}</div>",
            unsafe_allow_html=True,
        )
    st.write("")
    st.markdown("#### 📰 관련 뉴스 📰")
    news_results = get_naver_news(user_question)

    if news_results:
        for news in news_results:
            title = news['title'].replace("<b>", "").replace("</b>", "")
            link = news['link']
            st.markdown(f"🔗 [{title}]({link})", unsafe_allow_html=True)


In [ ]:
!ls

# streamlit 접속하기 및 연결 해제

In [ ]:
#개인 토큰 번호 입력
!ngrok authtoken [토큰값]

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&

In [ ]:
publ_url = ngrok.connect(addr="8501")

In [ ]:
publ_url

In [ ]:
!ps

In [ ]:
!kill [PID]

In [ ]:
ngrok.kill()